In [22]:
import numpy as np
import pandas as pd
import itertools as it

filename        = "Dataset_Covid_CE.csv"
k_values        = [2, 4, 8, 16]

def get_filename_out(k):
    return f"Dataset_Covid_CE_Anon_Sup_k{k}.csv"

def generate_csv(df, k):
    return df.to_csv(get_filename_out(k))

def is_k_anon(table, qt, k = 2):
    """
    para a tabela ser considerada k-anonima, todo elemento da tabela deve possuir pelo menos k outros elementos com a mesma combinaçao de semi-identificadores. ou seja, para k = 2 e qt = {Atr1, Atr2}, todo registro da tabela deve aparecer pelo menos 2 vezes com mesmo valor nos atributos "Atr1" e "Atr2".

    essa função agrupa os elementos da tabela pelos atributos semi-identificadores em qt e retorna False se algum grupo ocorre menos que k vezes, True se todo grupo ocorre ao menos k vezes.
    """

    groups = table.groupby(by=qt).size()
    for n_of_groups in groups:
        if n_of_groups < k: 
            return False
    return True

In [23]:
%%time
# limpeza

semi_id = [
    "municipioCaso",
    "sexoCaso",
    "dataNascimento",
    "resultadoFinalExame",
    "racaCor",
]

sensitive = [
    "comorbidadeCardiovascularSivep",
    "comorbidadeDiabetesSivep",
]

dtype = {
    "municipioCaso"                  : "str",
    "sexoCaso"                       : "str",
    "dataNascimento"                 : "str",
    "resultadoFinalExame"            : "str",
    "comorbidadeCardiovascularSivep" : "str",
    "comorbidadeDiabetesSivep"       : "str",
    "racaCor"                        : "str",
}

date_columns = [
    "dataNascimento",
]

date_parser = lambda x: pd.to_datetime(x, format="%Y-%m-%d", errors = 'coerce')

# pegar apenas os atributos desejados
# df = pd.read_csv(filename, nrows=100000, usecols=semi_id + sensitive, dtype=dtype, parse_dates=date_columns, date_parser=date_parser)
df = pd.read_csv(filename, usecols=semi_id + sensitive, dtype=dtype, parse_dates=date_columns, date_parser=date_parser)
df = df.dropna(how="all") # remover registros com todos os valores nulo
df = df.fillna("*") # padroniza valores nulos
df

Wall time: 11.7 s


,municipioCaso,sexoCaso,dataNascimento,resultadoFinalExame,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep,racaCor
19,SOBRAL,MASCULINO,2003-08-14 00:00:00,Negativo,*,*,Parda
20,PACAJUS,MASCULINO,1983-11-07 00:00:00,Negativo,*,*,Parda
21,HORIZONTE,FEMININO,1982-01-14 00:00:00,Negativo,*,*,*
25,FORTALEZA,MASCULINO,1992-03-12 00:00:00,Negativo,*,*,Parda
28,CAUCAIA,MASCULINO,1970-03-06 00:00:00,Negativo,*,*,Sem Informacao
...,...,...,...,...,...,...,...
1266272,FORTALEZA,FEMININO,1982-12-08 00:00:00,Positivo,*,*,Branca
1266273,FORTALEZA,FEMININO,1980-12-02 00:00:00,Negativo,*,*,Parda
1266274,FORTALEZA,FEMININO,1988-05-17 00:00:00,Provável,*,*,Branca
1266275,CRATEUS,FEMININO,1970-09-28 00:00:00,Negativo,*,*,Parda


In [24]:
is_k_anon(df, semi_id, k=2)

False

In [25]:
%%time

# 1. suprimir valores de cada semi-identificador que ocorrem < k vezes (pois não seria possível formar >= k grupos com eles)
def supress_unique_k_values(table, qt, k = 2):
    for col in qt:
        to_replace = []
        value_count = dict(table[col].value_counts(dropna=False))
        for value in value_count:
            if (value_count[value] < k):
                to_replace.append(value)
        table[col] = table[col].replace(to_replace, "*")
    return table

# df1 = supress_unique_k_values(df, semi_id, k=2)
# df1

Wall time: 0 ns


In [26]:
%%time
# 2. gerar tabela k = 2

# remove os grupos k >= 2 já formados da tabela
qt_5 = df[df.duplicated(subset=semi_id, keep=False)]
df2 = df.drop_duplicates(subset=semi_id, keep=False)

df_splices = [qt_5]

for n in [4, 3, 2, 1]:
    semi_id_combinations = it.combinations(semi_id, n)
    best_semi_id_combo = []
    duplicate_rows = []

    # para cada combinação de semi identificadores, checa quantos grupos k >= 2 formaria e pega a que gera mais
    for s in semi_id_combinations:
        rows = df2[df2.duplicated(subset=s, keep=False)]
        if len(duplicate_rows) < len(rows):
            duplicate_rows = rows.copy()
            best_semi_id_combo = s

    # pega o(s) atributo(s) que não aparece(m) na combinação e aplica supressão
    rm = list(set(semi_id) - set(best_semi_id_combo))
    for col in rm:
        duplicate_rows[col] = "*"

    # remove grupos recém formados da tabela original
    df_splices.append(duplicate_rows)
    df2 = df2.drop_duplicates(subset=best_semi_id_combo, keep=False)

df2.dataNascimento = "*"
df2.racaCor = "*"
df_splices.append(df2)

df2 = pd.concat(df_splices).sort_index()
df2

Wall time: 5.07 s


,municipioCaso,sexoCaso,dataNascimento,resultadoFinalExame,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep,racaCor
19,SOBRAL,MASCULINO,*,Negativo,*,*,Parda
20,PACAJUS,MASCULINO,1983-11-07 00:00:00,Negativo,*,*,Parda
21,HORIZONTE,FEMININO,*,Negativo,*,*,*
25,FORTALEZA,MASCULINO,1992-03-12 00:00:00,Negativo,*,*,Parda
28,CAUCAIA,MASCULINO,*,Negativo,*,*,Sem Informacao
...,...,...,...,...,...,...,...
1266272,FORTALEZA,FEMININO,*,Positivo,*,*,Branca
1266273,FORTALEZA,FEMININO,1980-12-02 00:00:00,Negativo,*,*,Parda
1266274,FORTALEZA,FEMININO,*,Provável,*,*,Branca
1266275,CRATEUS,FEMININO,1970-09-28 00:00:00,Negativo,*,*,Parda


In [27]:
is_k_anon(df2, semi_id, k=2)
# generate_csv(df2, k=2)

True

In [31]:
%%time
# 3. gerar tabela k = 4

k = 4
# df4 = supress_unique_k_values(df2, semi_id, k)
df4 = df2

need_supression = []
for name, group in df4.groupby(semi_id):
    if group.shape[0] < k:
        need_supression.append(group)

need_supression = pd.concat(need_supression).sort_index()
df4 = df4.drop(need_supression.index)

suppressed = [df4]

for n in [4, 3, 2, 1]:
    semi_id_combinations = it.combinations(semi_id, n)
    trows = []

    for s in semi_id_combinations:
        groups = need_supression.groupby(list(s))
        rows = []

        for name, group in groups:
            if group.shape[0] >= k:
                rm = list(set(semi_id) - set(s))
                for col in rm:
                    group[col] = "*"
                rows.append(group)

        if rows:
            rows = pd.concat(rows).sort_index()
            trows.append(rows)
            need_supression = need_supression.drop(rows.index)

    if trows:
        trows = pd.concat(trows).sort_index()
        suppressed.append(trows)

need_supression.dataNascimento = "*"
need_supression.racaCor = "*"
suppressed.append(need_supression)

df4 = pd.concat(suppressed).sort_index()
df4

Wall time: 20.5 s


,municipioCaso,sexoCaso,dataNascimento,resultadoFinalExame,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep,racaCor
19,SOBRAL,MASCULINO,*,Negativo,*,*,Parda
20,PACAJUS,MASCULINO,*,Negativo,*,*,Parda
21,HORIZONTE,FEMININO,*,Negativo,*,*,*
25,FORTALEZA,MASCULINO,1992-03-12 00:00:00,Negativo,*,*,*
28,CAUCAIA,MASCULINO,*,Negativo,*,*,Sem Informacao
...,...,...,...,...,...,...,...
1266272,FORTALEZA,FEMININO,*,Positivo,*,*,Branca
1266273,FORTALEZA,FEMININO,1980-12-02 00:00:00,*,*,*,Parda
1266274,FORTALEZA,FEMININO,*,Provável,*,*,Branca
1266275,CRATEUS,FEMININO,1970-09-28 00:00:00,Negativo,*,*,Parda


In [33]:
is_k_anon(df4, semi_id, k=4)
# generate_csv(df4, k=4)

In [35]:
%%time
# 3. gerar tabela k = 8

k = 8
df8 = df4

need_supression = []
for name, group in df8.groupby(semi_id):
    if group.shape[0] < k:
        need_supression.append(group)

need_supression = pd.concat(need_supression).sort_index()
df8 = df8.drop(need_supression.index)

suppressed = [df8]

for n in [4, 3, 2, 1]:
    semi_id_combinations = it.combinations(semi_id, n)
    trows = []

    for s in semi_id_combinations:
        groups = need_supression.groupby(list(s))
        rows = []

        for name, group in groups:
            if group.shape[0] >= k:
                rm = list(set(semi_id) - set(s))
                for col in rm:
                    group[col] = "*"
                rows.append(group)

        if rows:
            rows = pd.concat(rows).sort_index()
            trows.append(rows)
            need_supression = need_supression.drop(rows.index)

    if trows:
        trows = pd.concat(trows).sort_index()
        suppressed.append(trows)

need_supression.dataNascimento = "*"
need_supression.racaCor = "*"
suppressed.append(need_supression)

df8 = pd.concat(suppressed).sort_index()
df8

Wall time: 5.23 s


,municipioCaso,sexoCaso,dataNascimento,resultadoFinalExame,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep,racaCor
19,SOBRAL,MASCULINO,*,Negativo,*,*,Parda
20,PACAJUS,MASCULINO,*,Negativo,*,*,Parda
21,HORIZONTE,FEMININO,*,Negativo,*,*,*
25,FORTALEZA,MASCULINO,1992-03-12 00:00:00,Negativo,*,*,*
28,CAUCAIA,MASCULINO,*,Negativo,*,*,Sem Informacao
...,...,...,...,...,...,...,...
1266272,FORTALEZA,FEMININO,*,Positivo,*,*,Branca
1266273,FORTALEZA,FEMININO,*,*,*,*,Parda
1266274,FORTALEZA,FEMININO,*,Provável,*,*,Branca
1266275,CRATEUS,FEMININO,*,Negativo,*,*,Parda


In [37]:
is_k_anon(df8, semi_id, k=8)
# generate_csv(df8, k=8)

In [38]:
%%time
# 3. gerar tabela k = 16

k = 16
df16 = df8

need_supression = []
for name, group in df16.groupby(semi_id):
    if group.shape[0] < k:
        need_supression.append(group)

need_supression = pd.concat(need_supression).sort_index()
df16 = df16.drop(need_supression.index)

suppressed = [df16]

for n in [4, 3, 2, 1]:
    semi_id_combinations = it.combinations(semi_id, n)
    trows = []

    for s in semi_id_combinations:
        groups = need_supression.groupby(list(s))
        rows = []

        for name, group in groups:
            if group.shape[0] >= k:
                rm = list(set(semi_id) - set(s))
                for col in rm:
                    group[col] = "*"
                rows.append(group)

        if rows:
            rows = pd.concat(rows).sort_index()
            trows.append(rows)
            need_supression = need_supression.drop(rows.index)

    if trows:
        trows = pd.concat(trows).sort_index()
        suppressed.append(trows)

need_supression.dataNascimento = "*"
need_supression.racaCor = "*"
suppressed.append(need_supression)

df16 = pd.concat(suppressed).sort_index()
df16

Wall time: 2.58 s


,municipioCaso,sexoCaso,dataNascimento,resultadoFinalExame,comorbidadeCardiovascularSivep,comorbidadeDiabetesSivep,racaCor
19,SOBRAL,MASCULINO,*,Negativo,*,*,Parda
20,PACAJUS,MASCULINO,*,Negativo,*,*,Parda
21,HORIZONTE,FEMININO,*,Negativo,*,*,*
25,FORTALEZA,MASCULINO,*,Negativo,*,*,*
28,CAUCAIA,MASCULINO,*,Negativo,*,*,Sem Informacao
...,...,...,...,...,...,...,...
1266272,FORTALEZA,FEMININO,*,Positivo,*,*,Branca
1266273,FORTALEZA,FEMININO,*,*,*,*,Parda
1266274,FORTALEZA,FEMININO,*,Provável,*,*,Branca
1266275,CRATEUS,FEMININO,*,Negativo,*,*,Parda


In [41]:
is_k_anon(df16, semi_id, k=16)
# generate_csv(df16, k=16)

True